# Spatial Joins Exercises

Here\'s a reminder of some of the functions we have seen. Hint: they
should be useful for the exercises!

-   `sum(expression)`: aggregate to
    return a sum for a set of records
-   `count(expression)`: aggregate to
    return the size of a set of records
-   `ST_Area(geometry)` returns the
    area of the polygons
-   `ST_AsText(geometry)` returns WKT `text`
-   `ST_Contains(geometry A, geometry B)` returns the true if geometry A contains geometry B
-   `ST_Distance(geometry A, geometry B)` returns the minimum distance between geometry A and
    geometry B
-   `ST_DWithin(geometry A, geometry B, radius)` returns the true if geometry A is radius distance or less from geometry B
-   `ST_GeomFromText(text)` returns `geometry`
-   `ST_Intersects(geometry A, geometry B)` returns the true if geometry A intersects geometry B
-   `ST_Length(linestring)` returns the length of the linestring
-   `ST_Touches(geometry A, geometry B)` returns the true if the boundary of geometry A touches geometry B
-   `ST_Within(geometry A, geometry B)` returns the true if geometry A is within geometry B


Uncomment and run the following cell to install the required packages.


In [1]:
 %pip install duckdb leafmap lonboard
import duckdb
import leafmap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 667.8/667.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.5/208.5 kB 17.2 MB/

Download the [nyc_data.zip](https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip) dataset using leafmap. The zip file contains the following datasets. Create a new DuckDB database and import the datasets into the database. Each dataset should be imported into a separate table.

- nyc_census_blocks
- nyc_homicides
- nyc_neighborhoods
- nyc_streets
- nyc_subway_stations

# Sample Data

In [2]:
url = "https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip"
leafmap.download_file(url, unzip=True)

Downloading...
From: https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip
To: /content/nyc_data.zip
100%|██████████| 8.73M/8.73M [00:00<00:00, 35.2MB/s]


Extracting files...


'/content/nyc_data.zip'

# Connecting to DuckDB

In [3]:
con = duckdb.connect()

In [4]:
con.install_extension("spatial")
con.load_extension("spatial")

In [5]:
con.sql("""
    CREATE TABLE nyc_homicides AS SELECT * FROM 'nyc_homicides.shp';
    CREATE TABLE nyc_neighborhoods AS SELECT * FROM 'nyc_neighborhoods.shp';
    CREATE TABLE nyc_census_blocks AS SELECT * FROM 'nyc_census_blocks.shp';
    CREATE TABLE nyc_streets AS SELECT * FROM 'nyc_streets.shp';
    CREATE TABLE nyc_subway_stations AS SELECT * FROM 'nyc_subway_stations.shp';
""")

In [6]:
con.sql("SHOW TABLES;")

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ nyc_census_blocks   │
│ nyc_homicides       │
│ nyc_neighborhoods   │
│ nyc_streets         │
│ nyc_subway_stations │
└─────────────────────┘

In [7]:
con.sql("""

SELECT * FROM nyc_subway_stations

""").df()

,OBJECTID,ID,NAME,ALT_NAME,CROSS_ST,LONG_NAME,LABEL,BOROUGH,NGHBHD,ROUTES,TRANSFERS,COLOR,EXPRESS,CLOSED,geom
0,1.0,376.0,Cortlandt St,None,Church St,"Cortlandt St (R,W) Manhattan","Cortlandt St (R,W)",Manhattan,None,"R,W","R,W",YELLOW,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,2.0,2.0,Rector St,None,None,Rector St (1) Manhattan,Rector St (1),Manhattan,None,1,1,RED,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,3.0,1.0,South Ferry,None,None,South Ferry (1) Manhattan,South Ferry (1),Manhattan,None,1,1,RED,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,4.0,125.0,138th St,Grand Concourse,Grand Concourse,"138th St / Grand Concourse (4,5) Bronx","138th St / Grand Concourse (4,5)",Bronx,None,"4,5","4,5",GREEN,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,5.0,126.0,149th St,Grand Concourse,Grand Concourse,149th St / Grand Concourse (4) Bronx,149th St / Grand Concourse (4),Bronx,None,4,"2,4,5",GREEN,express,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,487.0,909.0,JFK Terminal 8,None,None,"JFK Terminal 8, Queens",JFK Terminal 8,Queens,None,None,None,AIR-BLUE,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
487,488.0,903.0,Federal Circle,Rental Car,None,"Federal Circle / Rental Car, Queens",Federal Circle / Rental Car,Queens,None,None,None,AIR-BLUE,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
488,489.0,902.0,Long Term Parking,None,None,"Long Term Parking, Queens",Long Term Parking,Queens,None,None,None,AIR-BLUE,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
489,490.0,901.0,Howard Beach,None,159th Ave,"Howard Beach, Queens",Howard Beach,Queens,None,None,A,AIR-BLUE,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [8]:
con.sql("""

SELECT * FROM nyc_neighborhoods

""").df()

,BORONAME,NAME,geom
0,Brooklyn,Bensonhurst,"[2, 4, 0, 0, 0, 0, 0, 0, 54, 71, 14, 73, 198, ..."
1,Manhattan,East Village,"[2, 4, 0, 0, 0, 0, 0, 0, 35, 215, 14, 73, 139,..."
2,Manhattan,West Village,"[2, 4, 0, 0, 0, 0, 0, 0, 161, 95, 14, 73, 212,..."
3,The Bronx,Throggs Neck,"[2, 4, 0, 0, 0, 0, 0, 0, 128, 232, 17, 73, 174..."
4,The Bronx,Wakefield-Williamsbridge,"[2, 4, 0, 0, 0, 0, 0, 0, 83, 85, 17, 73, 17, 2..."
...,...,...,...
124,Brooklyn,Red Hook,"[5, 4, 0, 0, 0, 0, 0, 0, 18, 0, 14, 73, 149, 8..."
125,Queens,Douglastown-Little Neck,"[2, 4, 0, 0, 0, 0, 0, 0, 251, 165, 19, 73, 76,..."
126,Queens,Whitestone,"[5, 4, 0, 0, 0, 0, 0, 0, 17, 75, 18, 73, 86, 2..."
127,Queens,Steinway,"[5, 4, 0, 0, 0, 0, 0, 0, 124, 87, 16, 73, 87, ..."


1. **What subway station is in \'Little Italy\'? What subway route is it on?**

In [9]:
con.sql("""
SELECT
    s.NAME AS station,
    s.ROUTES AS line
FROM
    nyc_subway_stations AS s,
    nyc_neighborhoods AS n
WHERE
    ST_Intersects(s.geom, n.geom)
    AND n.NAME = 'Little Italy'
""").df()

,station,line
0,Spring St,6


2. **What are all the neighborhoods served by the 6-train?** (Hint: The `routes` column in the `nyc_subway_stations` table has values like \'B,D,6,V\' and \'C,6\')


In [10]:
con.sql("""
SELECT
    DISTINCT n.NAME  -- 使用 DISTINCT 是为了去重，因为一个社区可能有多个6号线车站
FROM
    nyc_subway_stations AS s,
    nyc_neighborhoods AS n
WHERE
    ST_Intersects(s.geom, n.geom) -- 空间连接：车站落在社区里
    AND s.ROUTES LIKE '%6%';      -- 属性过滤：线路包含6号线
""").df()

,NAME
0,Gramercy
1,Murray Hill
2,Soundview
3,Upper East Side
4,Financial District
5,Hunts Point
6,East Harlem
7,Little Italy
8,Yorkville
9,Chinatown


3. **After 9/11, the \'Battery Park\' neighborhood was off limits for several days. How many people had to be evacuated?**

In [11]:
con.sql("""

SELECT * FROM nyc_census_blocks

""").df()

,BLKID,POPN_TOTAL,POPN_WHITE,POPN_BLACK,POPN_NATIV,POPN_ASIAN,POPN_OTHER,BORONAME,geom
0,360850009001000,97,51,32,1,5,8,Staten Island,"[2, 4, 0, 0, 0, 0, 0, 0, 55, 3, 13, 73, 151, 8..."
1,360850020011000,66,52,2,0,7,5,Staten Island,"[2, 4, 0, 0, 0, 0, 0, 0, 178, 58, 13, 73, 72, ..."
2,360850040001000,62,14,18,2,25,3,Staten Island,"[2, 4, 0, 0, 0, 0, 0, 0, 82, 227, 12, 73, 55, ..."
3,360850074001000,137,92,12,0,13,20,Staten Island,"[2, 4, 0, 0, 0, 0, 0, 0, 204, 85, 13, 73, 103,..."
4,360850096011000,289,230,0,0,32,27,Staten Island,"[2, 4, 0, 0, 0, 0, 0, 0, 107, 247, 12, 73, 7, ..."
...,...,...,...,...,...,...,...,...,...
38789,360050295001004,328,267,14,2,8,37,The Bronx,"[2, 4, 0, 0, 0, 0, 0, 0, 118, 130, 16, 73, 15,..."
38790,360050295002002,0,0,0,0,0,0,The Bronx,"[2, 4, 0, 0, 0, 0, 0, 0, 244, 138, 16, 73, 135..."
38791,360050419004001,0,0,0,0,0,0,The Bronx,"[2, 4, 0, 0, 0, 0, 0, 0, 78, 3, 17, 73, 85, 28..."
38792,360050255002001,480,96,96,20,12,256,The Bronx,"[2, 4, 0, 0, 0, 0, 0, 0, 4, 123, 16, 73, 154, ..."


In [12]:
con.sql("""

SELECT * FROM nyc_homicides

""").df()

,INCIDENT_D,BORONAME,NUM_VICTIM,PRIMARY_MO,ID,WEAPON,LIGHT_DARK,YEAR,geom
0,2008-01-01,Brooklyn,1,None,7,gun,D,2008,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,2008-01-04,Manhattan,1,None,14,gun,D,2008,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,2008-01-05,Queens,1,None,15,gun,D,2008,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,2008-01-04,Queens,1,None,16,knife,D,2008,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,2008-01-05,Queens,1,None,18,gun,D,2008,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...
3977,2010-10-11,The Bronx,1,None,4269,gun,None,2010,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3978,2010-10-06,The Bronx,1,None,4271,knife,None,2010,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3979,2011-07-26,The Bronx,1,None,4282,gun,None,2011,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3980,2011-07-28,The Bronx,1,None,4284,gun,None,2011,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [13]:
con.sql("""
SELECT
    SUM(c.POPN_TOTAL)
FROM
    nyc_census_blocks AS c
JOIN
    nyc_neighborhoods AS n
    ON ST_Intersects(c.geom, n.geom)
WHERE
    n.NAME = 'Battery Park';
""").df()

,sum(c.POPN_TOTAL)
0,17153.0


4. **What neighborhood has the highest population density (persons/km2)?**


In [14]:
con.sql("""
SELECT
    n.NAME,
    SUM(c.POPN_TOTAL) / (ST_Area(n.geom) / 1000000) AS popn_density
FROM
    nyc_neighborhoods AS n
JOIN
    nyc_census_blocks AS c
    ON ST_Intersects(n.geom, c.geom)
GROUP BY
    n.NAME, n.geom
ORDER BY
    popn_density DESC
LIMIT 2;
""").df()

,NAME,popn_density
0,North Sutton Area,68435.132838
1,East Village,50404.483413


When you're finished, you can check your answers [here](https://postgis.net/workshops/postgis-intro/joins_exercises.html).

# Ship-to-Ship Transfer Detection

Now for a less structured exercise. We're going to look at ship-to-ship transfers. The idea is that two ships meet up in the middle of the ocean, and one ship transfers cargo to the other. This is a common way to avoid sanctions, and is often used to transfer oil from sanctioned countries to other countries. We're going to look at a few different ways to detect these transfers using AIS data.

In [15]:
%pip install duckdb duckdb-engine jupysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.6/194.6 kB 12.0 MB/s eta 0:00:00


In [16]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [17]:
%%sql
INSTALL httpfs;
LOAD httpfs;
INSTALL spatial;
LOAD spatial;

,Success


## Step 1

Create a spatial database using the following AIS data:

https://storage.googleapis.com/qm2/casa0025_ships.csv

Each row in this dataset is an AIS 'ping' indicating the position of a ship at a particular date/time, alongside vessel-level characteristics.

It contains the following columns:
* `vesselid`: A unique numerical identifier for each ship, like a license plate
* `vessel_name`: The ship's name
* `vsl_descr`: The ship's type
* `dwt`: The ship's Deadweight Tonnage (how many tons it can carry)
* `v_length`: The ship's length in meters
* `draught`: How many meters deep the ship is draughting (how low it sits in the water). Effectively indicates how much cargo the ship is carrying
* `sog`: Speed over Ground (in knots)
* `date`: A timestamp for the AIS signal
* `lat`: The latitude of the AIS signal (EPSG:4326)
* `lon`: The longitude of the AIS signal (EPSG:4326)

Create a table called 'ais' where each row is a different AIS ping, with no superfluous information. Construct a geometry column.

Create a second table called 'vinfo' which contains vessel-level information with no superfluous information.

You can set a spatial index on each of these tables as follows:

`CREATE INDEX index_name ON table_name USING RTREE(geom);`

In [18]:
url = "https://storage.googleapis.com/qm2/casa0025_ships.csv"
leafmap.download_file(url, unzip=True)

Downloading...
From: https://storage.googleapis.com/qm2/casa0025_ships.csv
To: /content/casa0025_ships.csv
100%|██████████| 14.6M/14.6M [00:00<00:00, 76.6MB/s]


'/content/casa0025_ships.csv'

In [21]:
con.read_csv('casa0025_ships.csv').df()

,vesselid,vessel_name,vsl_descr,dwt,v_length,draught,sog,date,lat,lon,geom
0,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,5.2,2022-07-25 02:53:29,45.151777,36.513327,POINT (36.5133266666667 45.1517766666667)
1,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,0.7,2022-07-25 03:09:37,45.146487,36.520780,POINT (36.52078 45.1464866666667)
2,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,0.7,2022-07-25 03:13:58,45.146218,36.521965,POINT (36.521965 45.1462183333333)
3,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,0.1,2022-07-25 04:16:06,45.145058,36.522020,POINT (36.52202 45.1450583333333)
4,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,0.0,2022-07-25 05:20:17,45.144933,36.521848,POINT (36.5218483333333 45.1449333333333)
...,...,...,...,...,...,...,...,...,...,...,...
101323,217531,Zubeyde,roll on roll off with container capacity,5000.0,113.0,4.5,0.1,2022-08-10 14:16:47,45.091987,36.522157,POINT (36.5221566666667 45.0919866666667)
101324,217531,Zubeyde,roll on roll off with container capacity,5000.0,113.0,4.5,0.1,2022-08-10 14:43:48,45.091643,36.522213,POINT (36.5222133333333 45.0916433333333)
101325,217531,Zubeyde,roll on roll off with container capacity,5000.0,113.0,4.5,5.8,2022-08-10 15:04:28,45.100457,36.519397,POINT (36.5193966666667 45.1004566666667)
101326,217531,Zubeyde,roll on roll off with container capacity,5000.0,113.0,4.5,8.3,2022-08-23 06:06:51,45.087527,36.506987,POINT (36.5069866666667 45.0875266666667)


In [24]:
con.sql("""
CREATE TABLE ais AS
SELECT
    vesselid,
    date,
    sog,
    ST_Point(lon, lat) AS geom
FROM 'casa0025_ships.csv';
""")

In [25]:
con.sql("CREATE INDEX ais_geom_idx ON ais USING RTREE(geom);")

In [22]:
con.sql("""
CREATE TABLE vinfo AS
SELECT DISTINCT
    vesselid,
    vessel_name,
    vsl_descr,
    dwt,
    v_length,
    draught
FROM 'casa0025_ships.csv';
""")

In [26]:
con.sql("""

SELECT * FROM ais

""").df()

,vesselid,date,sog,geom
0,350053,2022-07-25 02:53:29,5.2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,350053,2022-07-25 03:09:37,0.7,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,350053,2022-07-25 03:13:58,0.7,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,350053,2022-07-25 04:16:06,0.1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,350053,2022-07-25 05:20:17,0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
...,...,...,...,...
101323,217531,2022-08-10 14:16:47,0.1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
101324,217531,2022-08-10 14:43:48,0.1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
101325,217531,2022-08-10 15:04:28,5.8,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
101326,217531,2022-08-23 06:06:51,8.3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [27]:
con.sql("""

SELECT * FROM vinfo

""").df()

,vesselid,vessel_name,vsl_descr,dwt,v_length,draught
0,256543,Omskiy 143,general cargo,3104.0,108.0,2.4
1,256543,Omskiy 143,general cargo,3104.0,108.0,2.6
2,301537,Omskiy 86,general cargo with container capacity,3201.0,108.0,3.4
3,296750,Omskiy 99,general cargo,3108.0,54.0,3.3
4,296750,Omskiy 99,general cargo,3108.0,54.0,3.0
...,...,...,...,...,...,...
1935,256640,Omskiy 129,general cargo,3174.0,108.0,2.5
1936,256640,Omskiy 129,general cargo,3174.0,108.0,3.2
1937,262746,Omskiy 133,general cargo,3070.0,108.0,2.6
1938,265324,Omskiy 14,general cargo,3174.0,108.0,2.4


## Step 2

Use a spatial join to identify ship-to-ship transfers in this dataset.
Two ships are considered to be conducting a ship to ship transfer IF:

* They are within 500 meters of each other
* For more than two hours
* And their speed is lower than 1 knot

Some things to consider: make sure you're not joining ships with themselves. Try working with subsets of the data first while you try different things out.

In [30]:
con.sql("""
CREATE OR REPLACE TEMP TABLE low_speed_ais AS
SELECT * FROM ais WHERE sog < 1;

SELECT
    a.vesselid AS ship_A,
    b.vesselid AS ship_B
FROM low_speed_ais a
JOIN low_speed_ais b ON a.vesselid < b.vesselid
    AND ST_DWithin(a.geom, b.geom, 500, TRUE) -- 使用GEOMETRY类型并指定距离为米，开启球面计算
WHERE
    a.date BETWEEN b.date - INTERVAL '2 hours' AND b.date + INTERVAL '2 hours'
GROUP BY
    ship_A, ship_B
HAVING
    epoch(MAX(a.date) - MIN(a.date)) > 7200; -- 7200秒即2小时
""").df()

BinderException: Binder Error: No function matches the given name and argument types 'ST_DWithin(GEOMETRY, GEOMETRY, INTEGER_LITERAL, BOOLEAN)'. You might need to add explicit type casts.
	Candidate functions:
	ST_DWithin(GEOMETRY, GEOMETRY, DOUBLE) -> BOOLEAN


In [32]:
con.sql("""
-- 1. 先在同一个连接中创建低速船只临时表
CREATE OR REPLACE TEMP TABLE low_speed_ais AS
SELECT * FROM ais WHERE sog < 1;

-- 2. 执行核心分析查询
SELECT
    a.vesselid AS ship_A,
    b.vesselid AS ship_B,
    MIN(a.date) AS start_time,
    MAX(a.date) AS end_time,
    age(MAX(a.date), MIN(a.date)) AS duration
FROM low_speed_ais a
JOIN low_speed_ais b
    ON a.vesselid < b.vesselid -- 避免重复配对
    AND ST_DWithin(a.geom, b.geom, 500.0, TRUE) -- 500米，TRUE表示开启球面计算
WHERE
    a.date BETWEEN b.date - INTERVAL '2 hours' AND b.date + INTERVAL '2 hours'
GROUP BY
    ship_A, ship_B
HAVING
    epoch(MAX(a.date) - MIN(a.date)) > 7200; -- 超过2小时（7200秒）
""").df()

BinderException: Binder Error: No function matches the given name and argument types 'ST_DWithin(GEOMETRY, GEOMETRY, DECIMAL(4,1), BOOLEAN)'. You might need to add explicit type casts.
	Candidate functions:
	ST_DWithin(GEOMETRY, GEOMETRY, DOUBLE) -> BOOLEAN


In [34]:
con.sql("""
-- 1. Create temporary table for low speed pings
CREATE OR REPLACE TEMP TABLE low_speed_ais AS
SELECT * FROM ais WHERE sog < 1;

-- 2. Identify transfers
SELECT
    a.vesselid AS ship_A,
    b.vesselid AS ship_B,
    MIN(a.date) AS start_time,
    MAX(a.date) AS end_time,
    age(MAX(a.date), MIN(a.date)) AS duration
FROM low_speed_ais a
JOIN low_speed_ais b
    ON a.vesselid < b.vesselid
    -- Use ST_Distance_Sphere to get distance in meters for 4326 geometries
    AND ST_Distance_Sphere(a.geom, b.geom) < 500
WHERE
    a.date BETWEEN b.date - INTERVAL '2 hours' AND b.date + INTERVAL '2 hours'
GROUP BY
    ship_A, ship_B
HAVING
    epoch(MAX(a.date) - MIN(a.date)) > 7200;
""").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,ship_A,ship_B,start_time,end_time,duration
0,231063,352014,2022-08-30 20:47:55,2022-08-31 23:58:25,1 days 03:10:30
1,255336,12935785,2022-07-16 00:03:24,2022-08-31 23:46:36,45 days 23:43:12
2,255336,10832059,2022-08-08 20:48:34,2022-08-31 23:46:36,23 days 02:58:02
3,255350,307087,2022-08-30 19:59:38,2022-08-31 23:10:33,1 days 03:10:55
4,10832059,12935785,2022-06-08 04:05:06,2022-08-31 23:03:54,83 days 18:58:48
...,...,...,...,...,...
1371,230124,265524,2022-06-02 11:55:21,2022-06-02 17:13:29,0 days 05:18:08
1372,215477,281220,2022-06-02 04:02:21,2022-06-02 12:52:26,0 days 08:50:05
1373,230124,279773,2022-06-01 16:17:57,2022-06-02 02:49:17,0 days 10:31:20
1374,162620,263073,2022-06-01 16:32:50,2022-06-02 02:17:52,0 days 09:45:02
